# 나머지 성호 님 것 feature + 행정구역, 인구수

## 1. 데이터, 패키지 불러오기

In [64]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer                          

from sklearn.compose import ColumnTransformer,make_column_transformer

from sklearn.decomposition import PCA

In [36]:
train = pd.read_csv("PJT002_train.csv", encoding = 'utf8')
test = pd.read_csv("PJT002_test.csv", encoding = 'utf-8')
validation = pd.read_csv("PJT002_validation.csv", encoding = 'utf8')

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2. 전처리

- 도시, 시골 여부 변수 추가

In [37]:
train['emd_nm'] = train['emd_nm'].fillna(value='blank')
train.loc[train['emd_nm'].str[5:8].str.endswith('시'), 'urban'] = 1
train.loc[train['emd_nm'].str[5:8].str.endswith('군'), 'urban'] = 0

In [38]:
test['emd_nm'] = test['emd_nm'].fillna(value='blank')
test.loc[test['emd_nm'].str[5:8].str.endswith('시'), 'urban'] = 1
test.loc[test['emd_nm'].str[5:8].str.endswith('군'), 'urban'] = 0

In [39]:
validation['emd_nm'] = validation['emd_nm'].fillna(value='blank')
validation.loc[validation['emd_nm'].str[5:8].str.endswith('시'), 'urban'] = 1
validation.loc[validation['emd_nm'].str[5:8].str.endswith('군'), 'urban'] = 0

## 3. null 값 확인

In [40]:
data = train[['emd_nm', 'hm_cnt','fr_sttn_dstnc', 'bldng_ar_prc', 'fr_wthr_fclt_dstnc', 'fr_mn_cnt', 'cctv_dstnc', 'cctv_in_100m', 'fr_wthr_fclt_in_100m', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc', 'ahsm_dstnc','no_tbc_zn_dstnc','bldng_cnt_in_50m', 'urban']]
data.head()

,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,cctv_dstnc,cctv_in_100m,fr_wthr_fclt_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,urban
0,경상남도 창녕군 창녕읍,17360.0,4547,NaN,133,137.0,112,0,0,1891,29231,11322,88,0,0.0
1,경상남도 의령군 가례면,1791.0,6388,122581.0,489,85.0,818,0,0,4533,20480,3369,7727,0,0.0
2,경상남도 창녕군 창녕읍,17285.0,3340,618105.0,143,137.0,165,0,0,277,31197,12451,72,14,0.0
3,경상남도 창원시의창구 대산면,7327.0,179,719542.0,1585,176.0,131,0,0,438,4270,407,508,11,1.0
4,경상남도 창녕군 창녕읍,17278.0,4822,NaN,603,137.0,686,0,0,1702,29778,12487,707,0,0.0


In [41]:
data.isnull().sum()

emd_nm                      0
hm_cnt                    701
fr_sttn_dstnc               0
bldng_ar_prc            21895
fr_wthr_fclt_dstnc          0
fr_mn_cnt                  41
cctv_dstnc                  0
cctv_in_100m                0
fr_wthr_fclt_in_100m        0
tbc_rtl_str_dstnc           0
sft_emrgnc_bll_dstnc        0
ahsm_dstnc                  0
no_tbc_zn_dstnc             0
bldng_cnt_in_50m            0
urban                       4
dtype: int64

## 4. imputation

### 1) hm_cnt

In [42]:
from datawig import SimpleImputer
from datawig.utils import random_split
from sklearn.metrics import f1_score, classification_report
df_train, df_test = random_split(train, split_ratios=[0.8, 0.2])
# Initialize a SimpleImputer model
imputer = SimpleImputer(
    input_columns=['urban', 'fr_sttn_dstnc','fr_wthr_fclt_dstnc', 'cctv_dstnc', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc', 'bldng_cnt_in_50m'],  # columns containing information about 
    output_column='hm_cnt',  # the column we'd like to impute values for
    output_path='imputer_model'  # stores model data and metrics
)

# Fit an imputer model on the train data
imputer.fit(train_df= df_train, num_epochs=50)

# Impute missing values and return original dataframe with predictions
predictions = imputer.predict(df_test)

2019-12-01 15:09:07,702 [INFO]  
========== start: fit model
2019-12-01 15:09:07,704 [WARNING]  Already bound, ignoring bind()
2019-12-01 15:09:10,486 [INFO]  Epoch[0] Batch [0-1332]	Speed: 7679.83 samples/sec	cross-entropy=10.696649	hm_cnt-accuracy=0.000000
2019-12-01 15:09:13,170 [INFO]  Epoch[0] Train-cross-entropy=9.462320
2019-12-01 15:09:13,173 [INFO]  Epoch[0] Train-hm_cnt-accuracy=0.000000
2019-12-01 15:09:13,175 [INFO]  Epoch[0] Time cost=5.467
2019-12-01 15:09:13,184 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2019-12-01 15:09:13,433 [INFO]  Epoch[0] Validation-cross-entropy=8.988470
2019-12-01 15:09:13,435 [INFO]  Epoch[0] Validation-hm_cnt-accuracy=0.000000
2019-12-01 15:09:16,146 [INFO]  Epoch[1] Batch [0-1332]	Speed: 7875.32 samples/sec	cross-entropy=9.826403	hm_cnt-accuracy=0.000000
2019-12-01 15:09:18,747 [INFO]  Epoch[1] Train-cross-entropy=9.001403
2019-12-01 15:09:18,749 [INFO]  Epoch[1] Train-hm_cnt-accuracy=0.000000
2019-12-01 15:09:18,750 [INFO] 

2019-12-01 15:10:30,973 [INFO]  Epoch[14] Train-cross-entropy=7.671125
2019-12-01 15:10:30,976 [INFO]  Epoch[14] Train-hm_cnt-accuracy=0.000000
2019-12-01 15:10:30,978 [INFO]  Epoch[14] Time cost=5.241
2019-12-01 15:10:30,988 [INFO]  Saved checkpoint to "imputer_model\model-0014.params"
2019-12-01 15:10:31,276 [INFO]  Epoch[14] Validation-cross-entropy=7.502658
2019-12-01 15:10:31,279 [INFO]  Epoch[14] Validation-hm_cnt-accuracy=0.000000
2019-12-01 15:10:33,911 [INFO]  Epoch[15] Batch [0-1332]	Speed: 8116.38 samples/sec	cross-entropy=8.152767	hm_cnt-accuracy=0.000000
2019-12-01 15:10:36,693 [INFO]  Epoch[15] Train-cross-entropy=7.639980
2019-12-01 15:10:36,695 [INFO]  Epoch[15] Train-hm_cnt-accuracy=0.000000
2019-12-01 15:10:36,697 [INFO]  Epoch[15] Time cost=5.415
2019-12-01 15:10:36,706 [INFO]  Saved checkpoint to "imputer_model\model-0015.params"
2019-12-01 15:10:37,026 [INFO]  Epoch[15] Validation-cross-entropy=7.475775
2019-12-01 15:10:37,029 [INFO]  Epoch[15] Validation-hm_cnt-ac

2019-12-01 15:11:51,405 [INFO]  Saved checkpoint to "imputer_model\model-0028.params"
2019-12-01 15:11:51,672 [INFO]  Epoch[28] Validation-cross-entropy=7.323438
2019-12-01 15:11:51,674 [INFO]  Epoch[28] Validation-hm_cnt-accuracy=0.000000
2019-12-01 15:11:54,046 [INFO]  Epoch[29] Batch [0-1332]	Speed: 9003.38 samples/sec	cross-entropy=7.675114	hm_cnt-accuracy=0.000000
2019-12-01 15:11:56,803 [INFO]  Epoch[29] Train-cross-entropy=7.172597
2019-12-01 15:11:56,806 [INFO]  Epoch[29] Train-hm_cnt-accuracy=0.000000
2019-12-01 15:11:56,809 [INFO]  Epoch[29] Time cost=5.132
2019-12-01 15:11:56,819 [INFO]  Saved checkpoint to "imputer_model\model-0029.params"
2019-12-01 15:11:57,137 [INFO]  Epoch[29] Validation-cross-entropy=7.286284
2019-12-01 15:11:57,139 [INFO]  Epoch[29] Validation-hm_cnt-accuracy=0.000000
2019-12-01 15:11:59,705 [INFO]  Epoch[30] Batch [0-1332]	Speed: 8324.53 samples/sec	cross-entropy=7.644367	hm_cnt-accuracy=0.000000
2019-12-01 15:12:02,424 [INFO]  Epoch[30] Train-cross-

In [43]:
train['hm_cnt'] = train['hm_cnt'].fillna(predictions['hm_cnt_imputed'])

In [44]:
train['hm_cnt'].isnull().sum()

569

### 2) fr_mn_cnt

In [45]:
from datawig import SimpleImputer
from datawig.utils import random_split
from sklearn.metrics import f1_score, classification_report
df_train, df_test = random_split(train, split_ratios=[0.8, 0.2])
# Initialize a SimpleImputer model
imputer = SimpleImputer(
    input_columns=['urban', 'fr_sttn_dstnc','fr_wthr_fclt_dstnc', 'cctv_dstnc', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc', 'bldng_cnt_in_50m'],  # columns containing information about 
    output_column='fr_mn_cnt',  # the column we'd like to impute values for
    output_path='imputer_model'  # stores model data and metrics
)

# Fit an imputer model on the train data
imputer.fit(train_df= df_train, num_epochs=10)

# Impute missing values and return original dataframe with predictions
predictions = imputer.predict(df_test)

2019-12-01 15:15:27,115 [INFO]  
========== start: fit model
2019-12-01 15:15:27,118 [WARNING]  Already bound, ignoring bind()
2019-12-01 15:15:29,856 [INFO]  Epoch[0] Batch [0-1332]	Speed: 7806.70 samples/sec	cross-entropy=2.439556	fr_mn_cnt-accuracy=0.015754
2019-12-01 15:15:32,673 [INFO]  Epoch[0] Train-cross-entropy=2.080105
2019-12-01 15:15:32,677 [INFO]  Epoch[0] Train-fr_mn_cnt-accuracy=0.008264
2019-12-01 15:15:32,680 [INFO]  Epoch[0] Time cost=5.558
2019-12-01 15:15:32,688 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2019-12-01 15:15:32,973 [INFO]  Epoch[0] Validation-cross-entropy=2.029376
2019-12-01 15:15:32,976 [INFO]  Epoch[0] Validation-fr_mn_cnt-accuracy=0.000000
2019-12-01 15:15:35,677 [INFO]  Epoch[1] Batch [0-1332]	Speed: 7905.92 samples/sec	cross-entropy=2.025287	fr_mn_cnt-accuracy=0.015754
2019-12-01 15:15:38,394 [INFO]  Epoch[1] Train-cross-entropy=1.922488
2019-12-01 15:15:38,396 [INFO]  Epoch[1] Train-fr_mn_cnt-accuracy=0.008264
2019-12-01 15:15:

In [46]:
train['fr_mn_cnt'] = train['fr_mn_cnt'].fillna(predictions['fr_mn_cnt_imputed'])

In [47]:
train['fr_mn_cnt'].isnull().sum()

32

### 3) urban

In [48]:
from datawig import SimpleImputer
from datawig.utils import random_split
from sklearn.metrics import f1_score, classification_report
df_train, df_test = random_split(train, split_ratios=[0.8, 0.2])
# Initialize a SimpleImputer model
imputer = SimpleImputer(
    input_columns=['hm_cnt', 'fr_sttn_dstnc', 'fr_wthr_fclt_dstnc', 'cctv_dstnc', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc', 'bldng_cnt_in_50m'],  # columns containing information about 
    output_column='urban',  # the column we'd like to impute values for
    output_path='imputer_model'  # stores model data and metrics
)

# Fit an imputer model on the train data
imputer.fit(train_df= df_train, num_epochs=10)

# Impute missing values and return original dataframe with predictions
predictions = imputer.predict(df_test)

2019-12-01 15:19:33,901 [INFO]  
========== start: fit model
2019-12-01 15:19:33,903 [WARNING]  Already bound, ignoring bind()
2019-12-01 15:19:36,751 [INFO]  Epoch[0] Batch [0-1332]	Speed: 7502.75 samples/sec	cross-entropy=8.373585	urban-accuracy=0.000750
2019-12-01 15:19:39,280 [INFO]  Epoch[0] Train-cross-entropy=6.918090
2019-12-01 15:19:39,282 [INFO]  Epoch[0] Train-urban-accuracy=0.000751
2019-12-01 15:19:39,284 [INFO]  Epoch[0] Time cost=5.377
2019-12-01 15:19:39,293 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2019-12-01 15:19:39,607 [INFO]  Epoch[0] Validation-cross-entropy=6.786742
2019-12-01 15:19:39,609 [INFO]  Epoch[0] Validation-urban-accuracy=0.000000
2019-12-01 15:19:42,401 [INFO]  Epoch[1] Batch [0-1332]	Speed: 7646.95 samples/sec	cross-entropy=6.180100	urban-accuracy=0.000750
2019-12-01 15:19:45,192 [INFO]  Epoch[1] Train-cross-entropy=5.745654
2019-12-01 15:19:45,193 [INFO]  Epoch[1] Train-urban-accuracy=0.000751
2019-12-01 15:19:45,196 [INFO]  Epoch

In [49]:
train['urban'] = train['urban'].fillna(predictions['urban_imputed'])

In [50]:
train['urban'].isnull().sum()

3

## 5. 최종 pipeline

In [73]:
categorical_feature = ['urban']
population_feature = ['hm_cnt', 'fr_mn_cnt']
price_feature = ['bldng_ar_prc']

categorical_transformer = Pipeline(steps = [
    ('smp', SimpleImputer(strategy = "most_frequent")),
    ('OneHotEncoer', OneHotEncoder(handle_unknown = 'ignore', sparse = False))
])

population_transformer = Pipeline([
    ('median', SimpleImputer(strategy = 'median'))
])

price_transformer = Pipeline(steps = [
    ('zero', SimpleImputer(strategy = 'median')),
    ('scaling', StandardScaler()),
    ('pca', PCA()),
])

processor = ColumnTransformer(
    transformers = [
    ('cat', categorical_transformer, categorical_feature),
    ('popu', population_transformer, population_feature),
    ('price', price_transformer, price_feature)
])


pipe = Pipeline(steps = [('processor', processor)
])                

In [74]:
pipe.fit_transform(train)

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.73600000e+04,  1.37000000e+02, -1.42597482e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.79100000e+03,  8.50000000e+01, -1.22772361e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.72850000e+04,  1.37000000e+02,  2.87853344e-02],
       ...,
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         3.01400000e+03,  1.39000000e+02, -1.42597482e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         3.75300000e+03,  1.39000000e+02, -1.42597482e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.60800000e+03,  1.37000000e+02, -1.42597482e-01]])

In [75]:
pipe.fit_transform(test)

array([[ 1.00000000e+00,  7.23500000e+03,  2.01000000e+02,
        -1.37845485e-01],
       [ 1.00000000e+00,  4.19800000e+03,  2.01000000e+02,
        -1.37845485e-01],
       [ 1.00000000e+00,  5.43540000e+04,  1.78000000e+02,
        -1.37845485e-01],
       ...,
       [ 1.00000000e+00,  8.42900000e+04,  2.01000000e+02,
         1.68933256e-01],
       [ 1.00000000e+00,  4.07030000e+04,  2.01000000e+02,
        -1.37845485e-01],
       [ 1.00000000e+00,  1.02610000e+04,  2.01000000e+02,
        -1.37845485e-01]])

In [76]:
pipe.fit_transform(validation)

array([[ 1.00000000e+00,  8.68600000e+03,  1.78000000e+02,
         7.26646268e-02],
       [ 1.00000000e+00,  7.77600000e+03,  1.78000000e+02,
         2.78414421e-01],
       [ 1.00000000e+00,  8.29240000e+04,  2.01000000e+02,
         1.40856709e-01],
       ...,
       [ 1.00000000e+00,  5.08350000e+04,  1.78000000e+02,
        -1.48400372e-01],
       [ 1.00000000e+00,  7.82520000e+04,  2.01000000e+02,
        -3.37037967e-01],
       [ 1.00000000e+00,  4.41780000e+04,  2.01000000e+02,
         5.35885883e-01]])